## D5a example/demo notebook
Demo/example notebook of the D5a 8-16 DAC module.

In [1]:
# Import SPI rack and S5i module
from spi_rack import SPI_rack
from D5a.D5a_module import D5a_module

import time

Open SPI rack connection and unlock (necessary after bootup of the controller module). Virtual COM port baud rate is irrelevant as is doesn't change the actual speed. Timeout can be changed, but 1 second is a good value.

In [2]:
COM_speed = 1e6 #Baud rate, doesn't do much
timeout = 1 #In seconds

spi_rack = SPI_rack('COM18', COM_speed, timeout)
spi_rack.unlock() #Unlock the controller to be able to send data

Create a new D5a module object at the correct (set) module address. By default the module resets the output voltages to 0 Volt. Before it does this it reads back the current value and if this value is non-zero it will slowly ramp it to zero. If `reset_voltages = False` then the output will not be changed. 

By default the number of DACs is set to 16.

In [28]:
D5a = D5a_module(spi_rack, 3, reset_voltages=True, num_dacs=16)

The output span of the DAC can be set in software to the following ranges:
* 0 to 4 Volt: range_4V_uni
* -4 to 4 Volt: range_4V_bi
* -2 to 2 Volt: range_2V_bi

Changing the span sets the smalles step size possible. The DACs used are 18-bit, which gives a smallest step size of:

$$\text{LSB} = \frac{\text{Span}}{2^{18}}$$

In the case of a $\pm$2V span this gives a LSB of: ~15 $\mu$V. You can get the current stepsize of a certain DAC with the `get_stepsize` function.

In [29]:
D5a.change_span_update(2, D5a.range_2V_bi)

stepsize =  D5a.get_stepsize(2)
print stepsize

The voltage can be set using the `set_voltage` function. If you want the output to be precisely equal to the set value, the voltage should be an integer multiple of the stepsize. Especially if sweeps are performed this is recommended. Otherwise the steps might nog be equidistant.

Optionally the output voltage might also be changed by setting a digital bit value ranging from 0 to $2^{18}-1$ using `change_value_update`.

In [33]:
D5a.set_voltage(2, 2)

In [32]:
D5a.change_value_update(2, 131054)

When done with everything, it's recommended to nicely close the connection with the SPI rack.

In [34]:
spi_rack.close()